In [1]:
import csv
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
from scipy import ndimage
from sklearn.linear_model import SGDClassifier

In [2]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [3]:
# Sti til filen. I denne Notebook er alt data samlet i en mappe og herefter splittet.
path = 'C:/Users/kvjkv/OneDrive - Aarhus Universitet/7. Semester/BML - Biomedical MachineLearning/prostate158/'
File = open(path+'train.csv')
type(File)
csvreader = csv.reader(File)
header = []
header = next(csvreader)
rows = []
with open(path+'train.csv') as File:
    csvreader = csv.reader(File)
    header = next(csvreader)
    for row in csvreader:
        rows.append(row)

Opsætning inden indlæsning.

In [4]:
def zoom_function(path1):
    img = nib.load(path1).get_fdata()
    desired_depth = 24
    desired_width = 270
    desired_height = 270

    actual_depth = img.shape[-1]
    actual_width = img.shape[0]
    actual_height = img.shape[1]

    depthfactor = 1/(actual_depth/desired_depth)
    widthfactor = 1/(actual_width/desired_width)
    heightfactor = 1/(actual_height/desired_height)

    newimg = ndimage.zoom(img, (widthfactor, heightfactor, depthfactor), order=1)
    return newimg

Indlæsning af træningsdata.

In [5]:
X = np.empty([159,270,270,24],dtype=float);
Y = np.ones(159);
pathPatientList = path+'train/';
dirs = os.listdir(pathPatientList)
val = rows[0][0];
i = 0;

for idx, x in enumerate(dirs):
    if int(x) is int(val):
        X[i] = zoom_function(path+rows[i][1])
        
        if os.path.isfile(pathPatientList + x + '/empty.nii.gz'):
            Y[i] = 0    
        
        if i < len(rows)-1:
            i+=1;
            val = rows[i][0];

In [6]:
X_norm = X.astype('float32') / 4096

print(X_norm.shape)
print(Y.shape)

(159, 270, 270, 24)
(159,)


In [7]:
## Split af data
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_norm, Y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=42)

print("Training set shape:", x_train.shape)
print("Training label shape:", y_train.shape)
print(type(y_train))
print("Validation set shape:", x_val.shape)
print("Validation label shape:", y_val.shape)
print(type(y_val))
print("Test set shape:", x_test.shape)
print("Test label shape:", y_test.shape)
print(type(y_test))

Training set shape: (127, 270, 270, 24)
Training label shape: (127,)
<class 'numpy.ndarray'>
Validation set shape: (16, 270, 270, 24)
Validation label shape: (16,)
<class 'numpy.ndarray'>
Test set shape: (16, 270, 270, 24)
Test label shape: (16,)
<class 'numpy.ndarray'>


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

model = Sequential()

# Convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(270, 270, 24,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))

# Output layer for binary classification (sigmoid activation for 0 to 1 range)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_val, y_val))

accuracy = model.evaluate(x_test, y_test)[1]
print("Test accuracy: {:.2f}%".format(accuracy * 100))

1/1 [==============================] - 3s 3s/step - loss: 0.6799 - accuracy: 0.5625
Test accuracy: 56.25%


As seen below, by adding a simple custom Stochastic gradient descent we have improved the accuracy

In [9]:
from tensorflow.keras.optimizers import SGD

model = Sequential()

# Convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(270, 270, 24,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))

# Output layer for binary classification (sigmoid activation for 0 to 1 range)
model.add(Dense(1, activation='sigmoid'))

# Trying to this time use a custom optimizer, still a SGD, however now we can tweak the hyper parameters.
custom_sgd = SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=custom_sgd, metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_val, y_val))

accuracy = model.evaluate(x_test, y_test)[1]
print("Test accuracy: {:.2f}%".format(accuracy * 100))

1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - accuracy: 0.3750
Test accuracy: 37.50%


Now we try and tweak the hyperparameters of the custom sgd, to see what happens when the accuracy.

In [10]:
model = Sequential()

# Convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(270, 270, 24,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))

# Output layer for binary classification (sigmoid activation for 0 to 1 range)
model.add(Dense(1, activation='sigmoid'))

# Trying to this time use a custom optimizer, still a SGD, however now we can tweak the hyper parameters.
custom_sgd = SGD(learning_rate=0.05)
model.compile(loss='categorical_crossentropy', optimizer=custom_sgd, metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_val, y_val))

accuracy = model.evaluate(x_test, y_test)[1]
print("Test accuracy: {:.2f}%".format(accuracy * 100))

1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - accuracy: 0.3750
Test accuracy: 37.50%


In [11]:
model = Sequential()

# Convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(270, 270, 24,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))

# Output layer for binary classification (sigmoid activation for 0 to 1 range)
model.add(Dense(1, activation='sigmoid'))

# Trying to this time use a custom optimizer, still a SGD, however now we can tweak the hyper parameters.
custom_sgd = SGD(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', optimizer=custom_sgd, metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_val, y_val))

accuracy = model.evaluate(x_test, y_test)[1]
print("Test accuracy: {:.2f}%".format(accuracy * 100))

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 0.3750
Test accuracy: 37.50%


So a higher learning rate seems to help. However we can also try making the learning rate gradually reduce every iteration to avoid overshooting as much when nearing a minimum, this is done by using the exponentialDecay function from keras, which does exactly what the name implies and decreases the initial learning rate exponentially over iterations.

In [13]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

model = Sequential()

# Convolutional layers
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(270, 270, 24,1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(158, activation='relu'))
model.add(Dropout(0.5))

# Output layer for binary classification (sigmoid activation for 0 to 1 range)
model.add(Dense(1, activation='sigmoid'))

# Still using a custom SGD however now a custom learning rate from keras, the exponentialdecay.
initial_learning_rate = 0.3
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate,decay_steps, decay_rate)

custom_sgd = SGD(learning_rate=lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer=custom_sgd, metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_val, y_val))

accuracy = model.evaluate(x_test, y_test)[1]
print("Test accuracy: {:.2f}%".format(accuracy * 100))

1/1 [==============================] - 3s 3s/step - loss: nan - accuracy: 0.3750
Test accuracy: 37.50%
